Create client

In [3]:
from label_studio_sdk import Client

LABEL_STUDIO_URL = 'http://localhost:8080'
API_KEY = '681842051079710a6b0ebce5ec56f746e3400c98'

ls = Client(url=LABEL_STUDIO_URL, api_key=API_KEY)

Create Image Classification project

In [4]:
project = ls.start_project(
    title='Project Created from SDK: Image Preannotation',
    label_config='''
    <View>
    <Image name="image" value="$image"/>
    <Choices name="image_class" toName="image">
        <Choice value="Cat"/>
        <Choice value="Dog"/>
    </Choices>
    </View>
    '''
)

Import tasks in basic [JSON Label Studio format](https://labelstud.io/guide/tasks.html#Basic-Label-Studio-JSON-format)

In [5]:
project.import_tasks(
    [{
        'data': {'image': 'https://data.heartex.net/open-images/train_0/mini/0045dd96bf73936c.jpg'},
        'predictions': [{
            'result': [{
                'from_name': 'image_class',
                'to_name': 'image',
                'type': 'choices',
                'value': {
                    'choices': ['Dog']
                }
            }],
            'score': 0.87
        }]
    }, {
        'data': {'image': 'https://data.heartex.net/open-images/train_0/mini/0083d02f6ad18b38.jpg'},
        'predictions': [{
            'result': [{
                'from_name': 'image_class',
                'to_name': 'image',
                'type': 'choices',
                'value': {
                    'choices': ['Cat']
                }
            }],
            'score': 0.65
        }]
    }]
)

[307035, 307036]

Or the same but simpler, indicating from which source preannotation results should be taken

In [6]:
project.import_tasks(
    [{'image': f'https://data.heartex.net/open-images/train_0/mini/0045dd96bf73936c.jpg', 'pet': 'Dog'},
    {'image': f'https://data.heartex.net/open-images/train_0/mini/0083d02f6ad18b38.jpg', 'pet': 'Cat'}],
    preannotated_from_fields=['pet']
)

[307037, 307038]

If you store everything in CSV files, you can do even better:

In [11]:
import pandas as pd
pd.read_csv('images.csv')

,image,pet
0,https://data.heartex.net/open-images/train_0/m...,Dog
1,https://data.heartex.net/open-images/train_0/m...,Cat


In [14]:
project.import_tasks('images.csv', preannotated_from_fields=['pet'])

[307039, 307040]

Now you can open http://localhost:8080 and go annotate your tasks...

When done, let's compute preannotation accuracy based on newly created ground truth dataset

In [ ]:
from evalme import get_agreement


print('Preannotation agreement scores:')

total_score = 0
n = 0
for task in project.tasks:
    score = get_agreement(task['annotations'][0], task['predictions'][0])
    print(f'{task["id"]} ==> {score}')
    total_score += score
    n += 1

print(f'Preannotation accuracy: {100 * total_score / n: .0f}%')